In [1]:
import numpy as np
import keras 
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as k

2026-01-18 21:19:54.206008: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 21:19:54.661919: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-18 21:19:56.144555: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
img_rows, img_cols=28, 28

if k.image_data_format() == 'channels_first':
   x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
   x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
   inpx = (1, img_rows, img_cols)

else:
   x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
   x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
   inpx = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [4]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [5]:
# Input
inpx = Input(shape=inpx, name="input_layer")

# Convolution layers
layer1 = Conv2D(
    32, (3, 3),
    activation='relu',
    name="conv_layer_1"
)(inpx)

layer2 = Conv2D(
    64, (3, 3),
    activation='relu',
    name="last_conv_layer"   # ⭐ IMPORTANT FOR GRAD-CAM
)(layer1)

# Pooling & dropout
layer3 = MaxPooling2D(
    pool_size=(3, 3),
    name="max_pool"
)(layer2)

layer4 = Dropout(
    0.5,
    name="dropout"
)(layer3)

# Dense layers
layer5 = Flatten(name="flatten")(layer4)

layer6 = Dense(
    250,
    activation='sigmoid',
    name="dense_1"
)(layer5)

layer7 = Dense(
    10,
    activation='softmax',
    name="output_layer"
)(layer6)

# Model
model = Model(inputs=inpx, outputs=layer7)


2026-01-18 21:19:57.854860: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
model.compile(
    optimizer=keras.optimizers.Adadelta(),
    loss=keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

history = model.fit(
    x_train,
    y_train,
    epochs=12,
    batch_size=500,
    validation_data=(x_test, y_test)
)


Epoch 1/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 24s 192ms/step - accuracy: 0.1022 - loss: 2.4476 - val_accuracy: 0.1010 - val_loss: 2.4406
Epoch 2/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 194ms/step - accuracy: 0.1022 - loss: 2.4316 - val_accuracy: 0.1010 - val_loss: 2.4247
Epoch 3/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 190ms/step - accuracy: 0.1022 - loss: 2.4160 - val_accuracy: 0.1010 - val_loss: 2.4094
Epoch 4/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 22s 187ms/step - accuracy: 0.1022 - loss: 2.4010 - val_accuracy: 0.1010 - val_loss: 2.3948
Epoch 5/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 192ms/step - accuracy: 0.1022 - loss: 2.3869 - val_accuracy: 0.1010 - val_loss: 2.3809
Epoch 6/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 23s 192ms/step - accuracy: 0.1022 - loss: 2.3733 - val_accuracy: 0.1010 - val_loss: 2.3676
Epoch 7/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 208ms/step - accuracy: 0.1022 - loss: 2.3606 - val_accuracy: 0.1010 - val_loss: 2.3551
Epoch 8/12
120/120 ━━━━━━━━━━━━━━━━━━━━ 25s 211ms/step - accuracy: 0.1022 - loss: 2

In [9]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [10]:

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_1 (Conv2D)           │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ last_conv_layer (Conv2D)        │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pool (MaxPooling2D)         │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 250)            │     1,024,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │         2,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,136,730 (11.97 MB)

 Trainable params: 1,045,576 (3.99 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,091,154 (7.98 MB)

In [11]:
model.save('mnist_model.h5')
print("Model saved successfully as mnist_model.h5")

Model saved successfully as mnist_model.h5


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# 1. Load Data (MNIST)
# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 2. Preprocessing
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Convert class vectors to binary class matrices (One-hot encoding)
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(f"x_train shape: {x_train.shape} - {x_train.shape[0]} samples")
print(f"x_test shape: {x_test.shape} - {x_test.shape[0]} samples")

# 3. Model Architecture (CNN + BatchNorm + Dropout)
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        
        # First Block
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(), # Added for stability
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Second Block
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Flatten and Dense
        layers.Flatten(),
        layers.Dropout(0.2), # Added to prevent overfitting
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

# 4. Compilation (Using your requested settings)
model.compile(
    optimizer=keras.optimizers.Adadelta(), # As requested
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# 5. Data Augmentation (Optional but recommended in your scope)
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(x_train)

# 6. Training
batch_size = 128
epochs = 24

print("Starting training...")
# We use fit_generator (flow) for data augmentation
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_test, y_test),
    verbose=1
)

# 7. Evaluation
score = model.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]:.4f}")

# 8. Save Model
model.save("mnist_model.h5")
print("Model saved to mnist_model.h5")

# 9. Visualize Training History (Bonus)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.savefig('training_history.png')
print("Training history plot saved.")

x_train shape: (60000, 28, 28, 1) - 60000 samples
x_test shape: (10000, 28, 28, 1) - 10000 samples


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,210 (137.54 KB)

 Trainable params: 35,018 (136.79 KB)

 Non-trainable params: 192 (768.00 B)

Starting training...
Epoch 1/24
469/469 ━━━━━━━━━━━━━━━━━━━━ 32s 66ms/step - accuracy: 0.1100 - loss: 3.8448 - val_accuracy: 0.1050 - val_loss: 2.5077
Epoch 2/24
469/469 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1445 - loss: 3.1603 - val_accuracy: 0.2118 - val_loss: 2.5619
Epoch 3/24
378/469 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.1788 - loss: 2.8698